# RNN

- 이미지 인식에 CNN이 있다면 자연어 인식에는 RNN이 있음
- RNN은 상태가 고정된 데이터를 처리하는 다른 신경망과는 달리 자연어 처리나 음성인식처럼 순서가 있는 데이터를 처리하는데 강점을 갖음

### 개념
- 학습한 결과를 다음단계의 학습에 이용하는 구조
- RNN에서 신경망을 셀(cell)이라고 하며 RNN은 이 셀을 여러개 중첩해서 심층 신경망을 구성
- 이런 구조로 인해 학습 데이터를 단계별로 구분하여 입력하여야 함

### MNIST 구현

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
# 옵션 설정
learning_rate = 0.001
total_epoch = 30
batch_size = 128

# RNN 은 순서가 있는 자료를 다루므로,
# 한 번에 입력받는 갯수와, 총 몇 단계로 이루어져있는 데이터를 받을지를 설정
# 이를 위해 가로 픽셀수를 n_input 으로, 세로 픽셀수를 입력 단계인 n_step 으로 설정
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10

In [3]:
# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.float32, [None, n_class])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 에 학습에 사용할 셀을 생성합니다
# 다음 함수들을 사용하면 다른 구조의 셀로 간단하게 변경할 수 있습니다
# BasicRNNCell,BasicLSTMCell,GRUCell
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

# RNN 신경망을 생성합니다
# 원래는 다음과 같은 과정을 거쳐야 하지만
# states = tf.zeros(batch_size)
# for i in range(n_step):
#     outputs, states = cell(X[[:, i]], states)
# ...
# 다음처럼 tf.nn.dynamic_rnn 함수를 사용하면
# CNN 의 tf.nn.conv2d 함수처럼 간단하게 RNN 신경망을 만들어줍니다.
# 겁나 매직!!
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# 결과를 Y의 다음 형식과 바꿔야 하기 때문에
# Y : [batch_size, n_class]
# outputs 의 형태를 이에 맞춰 변경해야합니다.
# outputs : [batch_size, n_step, n_hidden]
#        -> [n_step, batch_size, n_hidden]
#        -> [batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


In [4]:
# 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(total_epoch):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # X 데이터를 RNN 입력 데이터에 맞게 [batch_size, n_step, n_input] 형태로 변환
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('최적화 완료!')

Epoch: 0001 Avg. cost = 0.528
Epoch: 0002 Avg. cost = 0.243
Epoch: 0003 Avg. cost = 0.188
Epoch: 0004 Avg. cost = 0.157
Epoch: 0005 Avg. cost = 0.146
Epoch: 0006 Avg. cost = 0.136
Epoch: 0007 Avg. cost = 0.125
Epoch: 0008 Avg. cost = 0.111
Epoch: 0009 Avg. cost = 0.109
Epoch: 0010 Avg. cost = 0.106
Epoch: 0011 Avg. cost = 0.100
Epoch: 0012 Avg. cost = 0.096
Epoch: 0013 Avg. cost = 0.091
Epoch: 0014 Avg. cost = 0.087
Epoch: 0015 Avg. cost = 0.093
Epoch: 0016 Avg. cost = 0.079
Epoch: 0017 Avg. cost = 0.081
Epoch: 0018 Avg. cost = 0.082
Epoch: 0019 Avg. cost = 0.071
Epoch: 0020 Avg. cost = 0.073
Epoch: 0021 Avg. cost = 0.078
Epoch: 0022 Avg. cost = 0.077
Epoch: 0023 Avg. cost = 0.065
Epoch: 0024 Avg. cost = 0.064
Epoch: 0025 Avg. cost = 0.068
Epoch: 0026 Avg. cost = 0.068
Epoch: 0027 Avg. cost = 0.061
Epoch: 0028 Avg. cost = 0.067
Epoch: 0029 Avg. cost = 0.067
Epoch: 0030 Avg. cost = 0.060
최적화 완료!


In [5]:
# 결과 확인
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(mnist.test.images)
test_xs = mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))

정확도: 0.9686


### 단어 자동완성 구현
- 영문자 4개로 구성된 단어를 학습시켜 3글자만 주어지면 나머지 한글자를 추천하여 단어를 완성하는 코드
- dynamic_rnn의 sequence_length 옵션을 사용하면 가변 길이 단어 학습 가능
- 짧은 단어는 가장 긴 단어의 길이 만큼 뒷부분을 0으로 채우고, 해단 단어의 길이를 계산해 sequence_length로 넘겨주면 됨(batch_size만큼의 배열로)
- 하지만 코드가 복잡해지므로 고정길이 단어를 사용함

In [6]:
import tensorflow as tf
import numpy as np


char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

# one-hot 인코딩 사용 및 디코딩을 하기 위해 연관 배열 생성
# {'a': 0, 'b': 1, 'c': 2, ..., 'j': 9, 'k', 10, ...}
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 다음 배열은 입력값과 출력값으로 다음처럼 사용할 것임
# wor -> X, d -> Y
# woo -> X, d -> Y
seq_data = ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']

In [7]:
# 단어들을 학습에 사용할 수 있는 형식으로 변환해주는 유틸리티 함수 작성
def make_batch(seq_data):
    input_batch = []
    target_batch = []

    for seq in seq_data:
        # 입력값용으로 단어의 처음 세 글자의 알파벳 인덱스를 구한 배열을 만듦
        # 여기서 생성하는 input_batch 와 target_batch 는 알파벳 배열의 인덱스 번호
        input = [num_dic[n] for n in seq[:-1]]
        # 출력값용으로 마지막 글자의 알파벳 인덱스를 구함
        target = num_dic[seq[-1]]
        # 입력값을 one-hot 인코딩으로 변환
        input_batch.append(np.eye(dic_len)[input])
        # 지금까지 손실함수로 사용하던 softmax_cross_entropy_with_logits 함수는
        # label 값을 one-hot 인코딩으로 넘겨줘야 하지만,
        # 이 예제에서 사용할 손실 함수인 sparse_softmax_cross_entropy_with_logits 는
        # one-hot 인코딩을 사용하지 않으므로 index 를 그냥 넘겨줌
        target_batch.append(target)

    return input_batch, target_batch

In [8]:
# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 30
# RNN 을 구성하는 시퀀스의 갯수
n_step = 3
n_input = n_class = dic_len

In [9]:
# 신경망 모델 구성
X = tf.placeholder(tf.float32, [None, n_step, n_input])
Y = tf.placeholder(tf.int32, [None])

W = tf.Variable(tf.random_normal([n_hidden, n_class]))
b = tf.Variable(tf.random_normal([n_class]))

# RNN 셀을 생성
cell1 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
# 과적합 방지를 위한 Dropout 기법을 사용
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
# 여러개의 셀을 조합해서 사용하기 위해 셀을 추가로 생성
cell2 = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)

# 앞서 만든 셀들을 MultiRNNCell 함수를 사용하여 조합
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

# tf.nn.dynamic_rnn 함수를 이용해 순환 신경망, 즉 Deep RNN 생성
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

# 최종 결과는 one-hot 인코딩 형식으로 최종 출력층을 만듦
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
model = tf.matmul(outputs, W) + b

# 손실함수는 sparse_softmax_cross_entropy_with_logits
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
# 최적화 함수로 AdamOptimizer
optimizer = tf.train. AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [10]:
# 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={X: input_batch, Y: target_batch})

    print('Epoch:', '%04d' % (epoch + 1),'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 3.096714
Epoch: 0002 cost = 2.225318
Epoch: 0003 cost = 1.243832
Epoch: 0004 cost = 1.443959
Epoch: 0005 cost = 0.551919
Epoch: 0006 cost = 0.739340
Epoch: 0007 cost = 0.621329
Epoch: 0008 cost = 0.484964
Epoch: 0009 cost = 0.269593
Epoch: 0010 cost = 0.456929
Epoch: 0011 cost = 0.430645
Epoch: 0012 cost = 0.290627
Epoch: 0013 cost = 0.270127
Epoch: 0014 cost = 0.192583
Epoch: 0015 cost = 0.218338
Epoch: 0016 cost = 0.072985
Epoch: 0017 cost = 0.208441
Epoch: 0018 cost = 0.088683
Epoch: 0019 cost = 0.094963
Epoch: 0020 cost = 0.101590
Epoch: 0021 cost = 0.026482
Epoch: 0022 cost = 0.040599
Epoch: 0023 cost = 0.066047
Epoch: 0024 cost = 0.031698
Epoch: 0025 cost = 0.031522
Epoch: 0026 cost = 0.092112
Epoch: 0027 cost = 0.072789
Epoch: 0028 cost = 0.009336
Epoch: 0029 cost = 0.070365
Epoch: 0030 cost = 0.048173
최적화 완료!


In [11]:
# 결과 확인
# 레이블값이 정수이므로 예측값도 정수로 변경
prediction = tf.cast(tf.argmax(model, 1), tf.int32)
# one-hot 인코딩이 아니므로 입력값을 그대로 비교함
prediction_check = tf.equal(prediction, Y)
accuracy = tf.reduce_mean(tf.cast(prediction_check, tf.float32))

input_batch, target_batch = make_batch(seq_data)

predict, accuracy_val = sess.run([prediction, accuracy], feed_dict={X: input_batch, Y: target_batch})

predict_words = []

for idx, val in enumerate(seq_data):
    last_char = char_arr[predict[idx]]
    predict_words.append(val[:3] + last_char)

print('\n=== 예측 결과 ===')
print('입력값:', [w[:3] + ' ' for w in seq_data])
print('예측값:', predict_words)
print('정확도:', accuracy_val)


=== 예측 결과 ===
입력값: ['wor ', 'woo ', 'dee ', 'div ', 'col ', 'coo ', 'loa ', 'lov ', 'kis ', 'kin ']
예측값: ['word', 'wood', 'deep', 'dive', 'cold', 'cool', 'load', 'love', 'kiss', 'kind']
정확도: 1.0


### Sequence to Sequence
- 구글이 기계번역에 사용하는 신경망 모델. 줄여서 Seq2Seq라고도 함
- 순차적인 정보를 입력받는 신경망(RNN)과 출력하는 신경망을 조합한 모델
- 번역이나 챗봇 등 문장을 입력받아 다른 문장을 출력하는 프로그램에서 많이 사용함

### 구성
- 입력을 위한 신경망인 인코더와 출력을 위한 신경망인 디코더로 구성
- 인코더는 원문을, 디코더는 인코더가 번역한 결과물을 입력받음
- 그 후 디코더가 출력한 결과물을 번역된 결과물과 비교하면서 학습함

### 구현
- Seq2Seq 모델을 이용해 번역 프로그램 구현
- 네글자의 영단어를 입력받아 두 글자의 한 단어로 번역하는 프로그램

- 특수한 심볼이 필요
- 디코더에 입력이 시작됨을 알려주는 심볼
- 디코더의 출력이 끝났음을 알려주는 심볼
- 빈 데이터를 채울 때 사용하는 아무 의미가 없는 심볼

In [12]:
import tensorflow as tf
import numpy as np

# S: 디코딩 입력의 시작을 나타내는 심볼
# E: 디코딩 출력을 끝을 나타내는 심볼
# P: 현재 배치 데이터의 time step 크기보다 작은 경우 빈 시퀀스를 채우는 심볼
#    예) 현재 배치 데이터의 최대 크기가 4 인 경우
#       word -> ['w', 'o', 'r', 'd']
#       to   -> ['t', 'o', 'P', 'P']
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i, n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood', '나무'],
            ['game', '놀이'], ['girl', '소녀'],
            ['kiss', '키스'], ['love', '사랑']]

# 입력 단어와 출력 단어를 한 글자씩 떼어낸 뒤 배열로 만든 후 one-hot 인코딩 형식으로 만들어주는 유틸리티 함수 작성
# 데이터는 인코더의 입력값, 디코더의 입력값과 출력값 3개로 구성됨
def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만듦
        input = [num_dic[n] for n in seq[0]]
        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여줌
        output = [num_dic[n] for n in ('S' + seq[1])]
        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E 를 붙여줌
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
        # 출력값만 one-hot 인코딩이 아님 (sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [13]:
# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 100
n_class = n_input = dic_len

# 신경망 모델 구성
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

# 인코더 셀을 구성
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,dtype=tf.float32)

# 디코더 셀을 구성
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)

cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
Use keras.layers.dense instead.


In [14]:
# 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

print('최적화 완료!')

Epoch: 0001 cost = 3.716231
Epoch: 0002 cost = 2.545332
Epoch: 0003 cost = 1.364257
Epoch: 0004 cost = 1.279100
Epoch: 0005 cost = 0.718221
Epoch: 0006 cost = 0.412528
Epoch: 0007 cost = 0.530755
Epoch: 0008 cost = 0.331317
Epoch: 0009 cost = 0.354214
Epoch: 0010 cost = 0.263098
Epoch: 0011 cost = 0.200787
Epoch: 0012 cost = 0.067855
Epoch: 0013 cost = 0.286764
Epoch: 0014 cost = 0.085576
Epoch: 0015 cost = 0.079821
Epoch: 0016 cost = 0.075864
Epoch: 0017 cost = 0.057984
Epoch: 0018 cost = 0.029934
Epoch: 0019 cost = 0.025458
Epoch: 0020 cost = 0.098907
Epoch: 0021 cost = 0.017394
Epoch: 0022 cost = 0.018796
Epoch: 0023 cost = 0.044315
Epoch: 0024 cost = 0.049872
Epoch: 0025 cost = 0.006104
Epoch: 0026 cost = 0.014634
Epoch: 0027 cost = 0.025819
Epoch: 0028 cost = 0.006335
Epoch: 0029 cost = 0.010885
Epoch: 0030 cost = 0.006798
Epoch: 0031 cost = 0.004003
Epoch: 0032 cost = 0.004879
Epoch: 0033 cost = 0.002377
Epoch: 0034 cost = 0.002356
Epoch: 0035 cost = 0.007388
Epoch: 0036 cost = 0

In [15]:
# 번역 테스트
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어] 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미 없는 값인 P 값으로 채움
    seq_data = [word, 'P' * len(word)]

    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch size, time step, input] 으로 나오기 때문에,
    # 2번째 차원인 input 차원을 argmax 로 취해 가장 확률이 높은 글자를 예측 값으로 만듦
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction, feed_dict={enc_input: input_batch, dec_input: output_batch, targets: target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만듦
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만듦
    end = decoded.index('E')
    translated = ''.join(decoded[:end])

    return translated


print('\n=== 번역 테스트 ===')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))
print('love ->', translate('love'))
print('loev ->', translate('loev'))
print('abcd ->', translate('abcd'))


=== 번역 테스트 ===
word -> 단어
wodr -> 나무
love -> 사랑
loev -> 사랑
abcd -> 소녀
